In [ ]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
# %run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [ ]:
# def Neural_Network_Regression(X_train,X_test,y_train,y_test,filtered_data):
#        # Define the objective function for Bayesian optimization

#     def objective(params):
#         hidden_layer_size1, hidden_layer_size2, hidden_layer_size3, hidden_layer_size4, hidden_layer_size5, activation, alpha, learning_rate_init, tol, batch_size, momentum = params

#         hidden_layer_sizes = (hidden_layer_size1, hidden_layer_size2, hidden_layer_size3, hidden_layer_size4, hidden_layer_size5)

#         model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
#                              activation=activation,
#                              alpha=alpha,
#                              learning_rate_init=learning_rate_init,
#                              tol=tol,
#                              batch_size=batch_size,
#                              momentum=momentum,
#                              early_stopping=True,
#                              random_state=1,
#                              max_iter=1000)

#         model.fit(X_train_scaled, y_train_scaled)
#         predict_train_scaled = model.predict(X_train_scaled)
#         r2 = r2_score(y_train_scaled, predict_train_scaled)

#         return -r2  # Negate the r2 score for minimization

#     # Scale the features and target
#     scaler_X = StandardScaler()
#     scaler_y = StandardScaler()
#     X_train_scaled = scaler_X.fit_transform(X_train)
#     X_test_scaled = scaler_X.transform(X_test)
#     y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
#     y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

#     # Define the hyperparameter space
#     param_space = [
#         Integer(10, 100, name="hidden_layer_size1"),
#         Integer(10, 100, name="hidden_layer_size2"),
#         Integer(10, 100, name="hidden_layer_size3"),
#         Integer(10, 100, name="hidden_layer_size4"),
#         Integer(10, 100, name="hidden_layer_size5"),
#         Categorical(['relu', 'tanh', 'logistic'], name="activation"),
#         Real(1e-5, 1e-1, name="alpha"),
#         Real(1e-5, 1e-1, name="learning_rate_init"),
#         Real(1e-5, 1e-1, name="tol"),
#         Integer(32, 256, name="batch_size"),
#         Real(0.5, 0.9, name="momentum")
#     ]

#     # Perform Bayesian optimization
#     results = gp_minimize(objective, param_space, n_calls=50, random_state=0)    
#     # Extract the best hyperparameters
#     results.x[7] = round(results.x[7], 4)
#     print(results.x)
#     hidden_layer_size1, hidden_layer_size2, hidden_layer_size3, hidden_layer_size4, hidden_layer_size5, activation, alpha, learning_rate_init, tol, batch_size, momentum = results.x
#     hidden_layer_sizes = (hidden_layer_size1, hidden_layer_size2, hidden_layer_size3, hidden_layer_size4, hidden_layer_size5)
    
#     # Create and train the model with the best hyperparameters
#     best_model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
#                          activation=activation,
#                          alpha=alpha,
#                          learning_rate_init=learning_rate_init,
#                          tol=tol,
#                          batch_size=batch_size,
#                          momentum=momentum,
#                          early_stopping = True,
#                          random_state=1,
#                          max_iter=1000)
#     best_model.fit(X_train_scaled, y_train_scaled)

#     # Make predictions
#     predict_train_scaled = best_model.predict(X_train_scaled)
#     predict_train = scaler_y.inverse_transform(predict_train_scaled.reshape(-1, 1)).ravel()
#     predict_test_scaled = best_model.predict(X_test_scaled)
#     predict_test = scaler_y.inverse_transform(predict_test_scaled.reshape(-1, 1)).ravel()

#     # Create DataFrames for visualization
#     train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
#     test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

#     # Concatenate and sort the DataFrames
#     combined_df = pd.concat([train_df, test_df])
#     combined_df = combined_df.sort_index()
#     print(combined_df)

#     # Calculate and print R2 scores
#     r2_score_train = round(r2_score(y_train, predict_train), 2)
#     r2_score_test = round(r2_score(y_test, predict_test), 2)
#     print('r2 train', r2_score_train)
#     print('r2 test', r2_score_test)

#     if (v == 'dCn'):
#         unit = 'dCn'
#     else:
#         unit = 'pm_conc'
    
#     Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit)
#     qq_plot(test_df, v, unit)
#     # Return the model and the best hyperparameters
#     return best_model,results.x, r2_score_test
#     #Feature_Importance_Random_Forest_Regressor(best_model, filtered_data, v, unit)

In [1]:
def nn_bayesian_objective(X_train, y_train, scaler_X, scaler_y, hidden_layer_sizes, activation, max_iter):
    # Adjust hidden_layer_sizes to integer
    hidden_layer_sizes = int(hidden_layer_sizes)
    max_iter = int(max_iter)
    activation_func = 'relu' if activation > 0.5 else 'tanh'

    # Initialize and fit model
    model = MLPRegressor(hidden_layer_sizes=(hidden_layer_sizes,) * 5,
                         activation=activation_func,
                         random_state=1,
                         max_iter=max_iter)

    X_train_scaled = scaler_X.transform(X_train)
    y_train_scaled = scaler_y.transform(y_train.values.reshape(-1, 1)).ravel()
    model.fit(X_train_scaled, y_train_scaled)

    # Predict and calculate negative R2 score
    predict_train_scaled = model.predict(X_train_scaled)
    return -metrics.r2_score(y_train_scaled, predict_train_scaled)

def Neural_Network_Regression(X_train, X_test, y_train, y_test, filtered_data):
    scaler_X = StandardScaler().fit(X_train)
    scaler_y = StandardScaler().fit(y_train.values.reshape(-1, 1))

    # Bayesian Optimization
    optimizer = BayesianOptimization(
        f=lambda hidden_layer_sizes, activation, max_iter: nn_bayesian_objective(X_train, y_train, scaler_X, scaler_y, hidden_layer_sizes, activation, max_iter),
        pbounds={'hidden_layer_sizes': (10, 100), 'activation': (0, 1), 'max_iter': (200, 1000)},
        random_state=42,
    )

    optimizer.maximize(init_points=10, n_iter=25, acq='ei')

    # Train final model with best parameters
    best_params = optimizer.max['params']
    best_params['hidden_layer_sizes'] = int(best_params['hidden_layer_sizes'])
    best_params['max_iter'] = int(best_params['max_iter'])
    print("Optimal Parameters:",best_params)
    
    best_hidden_layer_sizes = best_params['hidden_layer_sizes']
    best_activation = 'relu' if best_params['activation'] > 0.5 else 'tanh'
    best_max_iter = best_params['max_iter']

    final_model = MLPRegressor(hidden_layer_sizes=(best_hidden_layer_sizes,) * 5,
                               activation=best_activation,
                               random_state=1,
                               max_iter=best_max_iter)

    X_train_scaled = scaler_X.transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.transform(y_train.values.reshape(-1, 1)).ravel()
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

    final_model.fit(X_train_scaled, y_train_scaled)
    
    # Predictions
    predict_train = scaler_y.inverse_transform(final_model.predict(X_train_scaled).reshape(-1, 1)).ravel()
    predict_test = scaler_y.inverse_transform(final_model.predict(X_test_scaled).reshape(-1, 1)).ravel()
    
    # Create DataFrames for visualization
    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenate and sort the DataFrames
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()
    print(combined_df)

    # Calculate and print R2 scores
    r2_score_train = round(r2_score(y_train, predict_train), 2)
    r2_score_test = round(r2_score(y_test, predict_test), 2)
    print('r2 train', r2_score_train)
    print('r2 test', r2_score_test)

    # if (v == 'dCn'):
    #     unit = 'dCn'
    # else:
    #     unit = 'pm_conc'
    
    # Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit)
    
    # qq_plot(test_df, v, unit)
    # plt.savefig('D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/qqplots/'+'qqplot_NN'+'_'+'v'+'.pdf')
    # # Return the model and the best hyperparameters
    # print('params',best_params)
    # print('model',final_model)
    return final_model,best_params, r2_score_test
#     #Feature_Importance_Random_Forest_Regressor(best_model, filtered_data, v, unit)